In [1]:
%pip install numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [4]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [5]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [6]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [7]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [8]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [9]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [10]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [11]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [12]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [13]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [14]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [15]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750], dtype=int64)

In [16]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [17]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [18]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [19]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [20]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [21]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


## 1

In [22]:
# # Importar las librerías necesarias
# from sklearn.feature_extraction.text import TfidfVectorizer
#
# # Instanciar el vectorizador TF-IDF
# tfidfvect = TfidfVectorizer()
#
# # Aplicar la vectorización a los documentos de entrenamiento
# X_train = tfidfvect.fit_transform(newsgroups_train.data)
#
# # Aplicar la vectorización a los documentos de prueba
# X_test = tfidfvect.transform(newsgroups_test.data)
#
# # Mostrar información sobre la matriz documento-término resultante
# X_train.shape


In [23]:
# # Volver a ejecutar el análisis de similitud sin ace_tools
#
# import random
# import pandas as pd
#
# # Seleccionar 5 documentos al azar del conjunto de entrenamiento
# random_idxs = random.sample(range(X_train.shape[0]), 5)
#
# # Diccionario para almacenar los resultados
# similarity_results = {}
#
# for idx in random_idxs:
#     # Obtener el documento original y su categoría
#     original_class = newsgroups_train.target_names[y_train[idx]]
#
#     # Medir la similitud coseno con todos los documentos de train
#     cossim = cosine_similarity(X_train[idx], X_train)[0]
#
#     # Obtener los índices de los 5 documentos más similares (excluyendo el mismo documento)
#     mostsim = np.argsort(cossim)[::-1][1:6]
#
#     # Obtener las clases de los documentos más similares
#     similar_classes = [newsgroups_train.target_names[y_train[i]] for i in mostsim]
#
#     # Almacenar los resultados
#     similarity_results[idx] = {
#         "original_class": original_class,
#         "most_similar_indices": mostsim,
#         "most_similar_classes": similar_classes,
#         "similarity_scores": cossim[mostsim]
#     }
#
# # Crear un DataFrame para mostrar los resultados
# df_results = []
# for idx, data in similarity_results.items():
#     for i in range(5):
#         df_results.append({
#             "Original Index": idx,
#             "Original Class": data["original_class"],
#             "Similar Index": data["most_similar_indices"][i],
#             "Similar Class": data["most_similar_classes"][i],
#             "Similarity Score": data["similarity_scores"][i]
#         })
#
# df_similarity = pd.DataFrame(df_results)
#
# # Mostrar resultados en consola
# print(df_similarity)



## **Análisis de los Documentos Similares**

### **Documento 650 (`soc.religion.christian`)**
- **Coherente:** La mayoría de los documentos similares pertenecen a la categoría `soc.religion.christian`, lo cual es esperado.
- **Inesperado:** La presencia de un documento de `alt.atheism` con alta similitud (0.4621) sugiere que hay términos compartidos en discusiones sobre religión.

✅ **Conclusión:** La similitud tiene sentido en su mayoría, aunque el documento de `alt.atheism` es un caso interesante.

---

### **Documento 2427 (`talk.politics.misc`)**
- **Coherente:** La mayoría de los documentos similares están en `talk.politics.misc` y `talk.politics.guns`, lo que es lógico.
- **Inesperado:** Un documento de `sci.med` con similitud (0.1655) indica que pudo haber términos compartidos sobre políticas de salud pública.

✅ **Conclusión:** En general, la similitud es razonable, aunque el documento de `sci.med` es un caso atípico.

---

### **Documento 3081 (`comp.sys.mac.hardware`)**
- **Totalmente coherente:** Todos los documentos similares pertenecen a la misma categoría `comp.sys.mac.hardware`, lo cual indica que el modelo está funcionando bien en este caso.

✅ **Conclusión:** La similitud está bien capturada en este caso.

---

### **Documento 9598 (`sci.crypt`)**
- **Coherente:** Tres de los cinco documentos similares pertenecen a `sci.crypt`, lo que tiene sentido.
- **Inesperado:** Un documento en `alt.atheism` (0.1846) es raro, probablemente debido a términos compartidos en debates filosóficos o técnicos.
- **Extraño:** La presencia de un documento de `comp.sys.ibm.pc.hardware` (0.1819) podría deberse a términos comunes en informática.

✅ **Conclusión:** La similitud es mayormente correcta, pero el documento de `alt.atheism` es extraño.

---

### **Documento 9343 (`soc.religion.christian`)**
- **Coherente:** Dos documentos de `soc.religion.christian` son los más similares.
- **Parcialmente inesperado:** La similitud con documentos de `talk.politics.mideast`, `talk.politics.misc` y `talk.politics.guns` sugiere que los temas de religión y política pueden solaparse.

✅ **Conclusión:** La similitud es razonable, pero la conexión con documentos de política sugiere que en este caso la religión se está discutiendo en un contexto más político.

---

## **Conclusión General**
- **Casos correctos:**
  - `comp.sys.mac.hardware` fue identificado con alta precisión.
  - `soc.religion.christian` también tuvo buenas similitudes con documentos de la misma categoría.

- **Casos parcialmente correctos:**
  - `talk.politics.misc` y `sci.crypt` incluyen documentos de categorías inesperadas.
  - `talk.politics.mideast` y `soc.religion.christian` están relacionados en algunos contextos.

- **Casos inesperados:**
  - La presencia de `alt.atheism` en `soc.religion.christian` y `sci.crypt` puede ser por términos en común en debates filosóficos o técnicos.
  - `sci.med` apareciendo en `talk.politics.misc` puede indicar discusiones sobre políticas de salud.

---

## **Mejoras para el Modelo**
- **Eliminar términos comunes:** Excluir palabras compartidas en múltiples categorías (`stop words` adicionales).
- **Usar embeddings más avanzados:** Modelos como `Word2Vec`, `FastText` o `BERT` podrían capturar mejor el significado de los textos.
- **Probar otras métricas de similitud:** Además de la similitud coseno, se pueden probar técnicas como `LSA` o `UMAP` para encontrar relaciones más precisas.

---

✅ **¡Análisis completado!** 🚀
